In [ ]:
import pandas as pd

In [ ]:
path = r'C:\Users\ASUS\Desktop\龚华天\文件\OD-matrix.csv'
data = pd.read_csv(path,engine='python')

In [ ]:
data['total_cost'] = round(data['total_cost'],2)
G = []
for i in data.index:
    G.append((data['total_cost'][i],data['origin_id'][i],data['destination_id'][i]))

In [ ]:
from networkx.utils import UnionFind
import copy
import heapq
from collections import namedtuple


Partition = namedtuple('Partition', ['included', 'excluded'])


def getGraphCost(graph):
    '''Returns the total sum of weights of edges in a graph'''
    return sum(e[0] for e in graph)

def kruskal(edges):
    '''Returns the mst'''
    edges.sort()
    
    subtrees = UnionFind()
    solution = []

    for edge in edges:
        _, u, v = edge
        if subtrees[u] != subtrees[v]:
            solution.append(edge)
            subtrees.union(u, v)

    return solution

class IncreasingMST():
    def __init__(self, edges):
        self.edges = edges
        self.edges.sort()  # Sort edges by weight
        self.order = self.__getNbVertices(edges)
    
    @classmethod
    def __getNbVertices(self, edges):
        vertices = set()
        for c, u, v in edges:
            vertices.add(u)
            vertices.add(v)
        return len(vertices)
    
    @classmethod
    def __partition(self , mstEdges , p):
        '''
        Given an MST and a __partition P, P partitions the vertices of the MST.
        The union of the set of partitions generated here with the last MST 
        is equivalent to the last __partition as an input.
        '''
 
        p1 = copy.deepcopy(p)
        p2 = copy.deepcopy(p)
 
        partitions = []
        
        # Find open edges : they are in MST but are not included nor excluded 
        open_edges = [edge for edge in mstEdges if edge not in p.included 
                      and edge not in p.excluded]    
 
        for e  in open_edges :
            p1.excluded.append(e)
            p2.included.append(e)
            
            partitions.append(p1)
            p1 = copy.deepcopy(p2)
 
        return partitions
    
    def __kruskalMST(self, P):
        '''
        Returns the MST of a graph contained in the __partition P.
        Returns None if there is not any.
        P [0] -> edges included, P [1] -> edges excluded
        '''

        # Initialize the subtrees for Kruskal Algorithm with included edges
        subtrees = UnionFind()
        for c, u , v in P.included :
            subtrees.union(u , v)
        
        # Find open Edges
        edges = [e for e in self.edges if e not in P.included and e not in P.excluded]
        
        # Create a solution tree with the list of included edges from the __partition
        tree = list(P.included)
        
        # Add edges connecting vertices not connected yet, until we get to
        # A solution, or discover that you can not connect all vertices
        
        # Apply Kruskal on open edges
        for edge in edges :
            c, u , v = edge
            if subtrees [u] != subtrees [v]:
                tree.append(edge)
                subtrees.union(u, v)
        
        if self.order == self.__getNbVertices(tree) and \
        len(tree) == self.order - 1:
            return tree
        else:
            return None
    
    def mst_iter(self):
        '''Minimum spanning tree iterator : yields the MST in order of their cost
        Warning this quickly becomes computer intensive both on CPU and memory'''
        mst = kruskal(self.edges)

        mstCost = getGraphCost(mst)
        List = [(mstCost, Partition([], []), mst)]
        
        # While we have a __partition
        while len(List):
            # Get __partition with the smallest spanning tree
            # and remove it from the list
            cost, partition, tree = heapq.heappop(List)
            
            # Yield MST result
            yield tree
               
            edges = tree
            
            # Partition previous partition 
            newPartitions = self.__partition(edges, partition)
            for p in newPartitions:
                tree = self.__kruskalMST(p)
                if tree:
                    # print 'isTree', tree
                    cost = getGraphCost(tree)
                    heapq.heappush(List, (cost, p, tree))

In [ ]:
if  __name__ == '__main__' :
    
    print(len(G))
    iMST = IncreasingMST(G)

In [ ]:
variance = open("variance.dat", "w")
cost = open("cost.dat", "w")
tree_all = open("tree_all.dat", "w")

import numpy as np
NA = 0

for tree in iMST.mst_iter():
        inve = []
        for i in tree:
            inve.append(i[1])
            inve.append(i[2])
#         print(len(inve))
        a = []
        myset = set(inve) 
        for item in myset:
            a.append(inve.count(item))
        if NA<10000:
            
                va = np.var(a)
#                 print("variance: %f " %(va))

#                 print(tree)
    #             print map(id, tree) # Edges share the same memory space
                cos = getGraphCost(tree)
                print("variance: %f " %(va),'Cost =', cos)

                variance.write(str(va) + "\n")
                cost.write(str(cos) + "\n")
#                 tree_all.write(str(tree) + "\n")      
        else:
            break
        
        NA+=1
        print('数：'+ str(NA))
variance.close()
cost.close()
tree_all.close()